In [3]:
import pandas as pd
import numpy as np

In [4]:
# load dataset
bknote = pd.read_csv('data_banknote_authentication.txt',header = None).to_numpy()


In [5]:
def train_test_split(data,test_size):
    """ function that takes data (2d-array) and 
        test size (percentage)
        and returns the two data sets train and test
        test rows are selected randomly
    """
    nr_rows = data.shape[0]
    np.random.seed(3)
    test_rows = np.random.choice(nr_rows, size=round((test_size*nr_rows)), replace=False)
    test_ar = data[test_rows, :]
    train_ar = np.delete(data, test_rows, axis=0)
    return train_ar,test_ar
#train_test_split(bknote,0.3)

In [6]:
def sameLabel(data):
    """This function checks if all train points belong in the same class
        argument: data, 2d-array
        return : boolean 
    """
    all_classes = np.unique(data[:,-1])
    if(len(all_classes)>=2):
        return False
    else:
        return True

In [8]:
def allocate(data):
    """ For the given data, this function returns 
        the most commonly occuring class
    """
    labels,occurrence = np.unique(data[:,-1], return_counts=True)
    return labels[occurrence.argmax()]

#allocate(bknote[700:,])



In [28]:
def split_points(data):
    """ limit potential split points by considering only cases where 
        class lavel changes between to adjacent points
    """
    split_points = {}
    nr_columns = data.shape[1]
    for i in range(nr_columns-1):
        split_points[i] = []
        unique_points = np.unique(data[:,i])
        for k in range(1,len(unique_points)):
            potential_points = (unique_points[k]+unique_points[k-1])/2
            split_points[i].append(potential_points)
    return split_points
#split_points(bknote)



In [29]:
def separate_data(data,split_attribute,split_point):
    """ This function separates the data by an attribute at 
        a given split point, returns two 2d arrays, one left
        to the split point and one to the right
    """
    column = data[:,split_attribute]
    bool_ar_left = column <= split_point
    bool_ar_right = column > split_point
    left = data[bool_ar_left,]
    right = data[bool_ar_right,]
    return left, right
#separate_data(bknote,2,bknote[:,0].mean())[0]
#separate_data(bknote,0,bknote[:,0].mean())[1].shape

In [30]:
def entropy(data):
    #calculate entropy: information content
    label_counts = np.unique(data[:,-1],return_counts = True)[1]
    proportions = label_counts/label_counts.sum()
    entropy = np.sum(-proportions*np.log2(proportions))
    return entropy

#entropy(bknote) 


In [31]:
def gini_index(data):
    #calculate impurity
    label_counts = np.unique(data[:,-1],return_counts = True)[1]
    proportions = label_counts/label_counts.sum()
    gini = 1-np.sum(proportions**2)
    return gini
#gini_index(bknote)

In [32]:
def inf_gain(data,left,right, impurity_measure):
    if impurity_measure == 'entropy':
        # calculate the information at a given node
        entropy_before = entropy(data)
        prop_left = left.shape[0]/data.shape[0]
        prop_right = right.shape[0]/data.shape[0]
        entropy_after = prop_left*entropy(left) + prop_right*entropy(right)
        inf_gain = entropy_before - entropy_after
        return inf_gain
    elif impurity_measure == 'gini':
        gini_before = gini_index(data)
        prop_left = left.shape[0]/data.shape[0]
        prop_right = right.shape[0]/data.shape[0]
        gini_after = prop_left*gini_index(left) + prop_right*gini_index(right)
        inf_gain = gini_before - gini_after
        return inf_gain
        
#inf_gain(bknote,separate_data(bknote,0,bknote[:,0].mean())[0],separate_data(bknote,0,bknote[:,0].mean())[1],'gini')

In [33]:
def best_possible_split(data,impurity_measure):
    """ This function, given data, determines the best split 
        by iterating over possible splits, and returns the split
        attribute and split point
    """
    max_infgain = -1 # since information gain ranges between 0 and 1
    possible_splits = split_points(data)
    for c_index in possible_splits:
        for value in possible_splits[c_index]:
            left,right = separate_data(data,c_index,value)
            infgain = inf_gain(data,left,right,impurity_measure)
            
            if infgain > max_infgain:
                max_infgain = infgain
                split_col = c_index
                sp_point = value
    return split_col,sp_point
#best_possible_split(bknote,'gini')

In [34]:
def learn_pre_pruning(data,impurity_measure = 'entropy',pruning=False):
    """ This reccursive function  learns a decision tree classifier from a data matrix X
        and a label vector y.
        Both X and y are needed to be numppy arrays
    """
    # since all the helper fucntions take an entire array
    #y = np.transpose([list(y)])
    #data = np.append(X, y, axis=1) 
    # simplest case
    if sameLabel(data):
        classification = allocate(data)
        return classification
    else:
        possible_splits = split_points(data)
        best_split = best_possible_split(data,impurity_measure)
        left,right = separate_data(data,best_split[0],best_split[1])
        
        split_cond = f"{best_split[0]} <= {best_split[1]}"
        sub_tree = {split_cond:[]}
        cond_true = learn(left,impurity_measure)
        cond_false = learn(right,impurity_measure)

        sub_tree[split_cond].append(cond_true)
        sub_tree[split_cond].append(cond_false)        
        
        return sub_tree


In [35]:
def classifier(row,tree):
    """ This function classifies data  the class they belong given a fitted tree
        Argument: a row
        return: predicted label for a given row
    """
    split_condition = list(tree.keys())[0]
    split_attribute = int(split_condition.split()[0])
    sp_point = float(split_condition.split()[2])
    
    if row[split_attribute] <= sp_point:
        answer = tree[split_condition][0]
    else:
        answer = tree[split_condition][1]
        
    if not isinstance(answer,dict):
        return answer
    else:
        rest_of_tree = answer
        return classifier(row,rest_of_tree)
    
#classifier(bknote[900,:],tree)

In [36]:
def predict(data,tree):
    """ Main function to predict classes after fitting a model,"""
    pred = list()
    for i in range(data.shape[0]):
        pred.append(classifier(data[i,:],tree))
    return pred
#predict(bknote[0:5],tree)       

In [37]:
def after_pruning(tree,train,val):
    """ helper function to the pruned function"""
    labels,counts = list(np.unique(train[:,-1],return_counts = True))
    leaf = labels[list(counts).index(max(counts))]
    # now check which makes the most errors
    errors_leaf = sum(val[:,-1] != leaf)
    errors_at_node = sum(val[:,-1] != predict(val,tree))
    if errors_leaf <= errors_at_node:
        return leaf
    else:
        return tree
    

In [38]:
def pruned(tree,train,val):
    """ Function that prunes a given tree, given training 
        and validation sets. predicts a leaf node for each 
        tree node that perforsm less than or equal to the 
        majority class, returns a pruned tree
    """
    split_condition = list(tree.keys())[0]
    yes, no = tree[split_condition]
    # stopping 
    if not isinstance(yes,dict) and not isinstance(no,dict):
        return after_pruning(tree,train,val)  
    else:
        split_attribute = int(split_condition.split()[0])
        sp_point = float(split_condition.split()[2])
        train_left,train_right = separate_data(train,split_attribute,sp_point)
        val_left, val_right = separate_data(val,split_attribute,sp_point)
        
        if isinstance(yes,dict):
            tree[split_condition][0] = pruned(yes,train_left,val_left)
        if isinstance(no,dict):
            tree[split_condition][1] = pruned(no,train_right,val_right)
            
        return after_pruning(tree,train,val)



In [45]:
def learn(data,impurity_measure,pruning = False,val_size = 0.15):
    tree = learn_pre_pruning(data,impurity_measure)
    if pruning == True:
        train,validate = train_test_split(data,val_size)
        return pruned(tree,train,validate)
    else:
        return tree

In [46]:
def calc_accuracy(labels,predicted_labels):
    """ Calculate prediction accuracy given predicted and true labesl"""
    #calculate prediction accuracy
    mis_classified = 0
    if not len(labels) == len(predicted_labels):
        return 'true values and predicted ones differ in size'
    else:
        for i in range(len(labels)):
            if labels[i] != predicted_labels[i]:
                mis_classified +=1
    return (mis_classified/len(labels))

# Now that the learn function is ready, use it to train a classifier on the banknote dataset. 

#start by splitting the data into training,validation and test set.

In [48]:
train, test = train_test_split(bknote,0.15)
training_set, validation_set = train_test_split(train,0.15)

In [52]:
# the different trees 
tree_unpruned_gini = learn(training_set,'gini')
tree_unpruned_entropy = learn(training_set,'entropy')
tree_pruned_gini = learn(training_set,'gini',pruning = True)
tree_pruned_entropy = learn(training_set,'entropy',pruning = True)

In [63]:
trees = {'gini':[tree_unpruned_gini,tree_pruned_gini],'entropy':[tree_unpruned_entropy,tree_pruned_entropy]}
for key in trees.keys():
    print(f"The validation accuracy  for the unpruned tree by {key} is: {1-calc_accuracy(predict(validation_set,trees[key][0]),validation_set[:,-1])}")
    print(f"The validation accuracy  for the pruned tree by {key} is:  {1-calc_accuracy(predict(validation_set,trees[key][1]),validation_set[:,-1])}")
    

The validation accuracy  for the unpruned tree by gini is: 0.9771428571428571
The validation accuracy  for the pruned tree by gini is:  0.9428571428571428
The validation accuracy  for the unpruned tree by entropy is: 0.9828571428571429
The validation accuracy  for the pruned tree by entropy is:  0.9771428571428571


In [64]:
# the unpruned tree where entropy is the the impurity measure 
# performs best on the validation set
print(f"The test accuracy  for the chosen tree is: {1-calc_accuracy(predict(test,tree_unpruned_entropy),test[:,-1])}")



The test accuracy  for the chosen tree is: 0.9805825242718447


# comparison with existing implementation

In [67]:
from sklearn.tree import DecisionTreeClassifier

In [81]:
clf = DecisionTreeClassifier()
clf = clf.fit(training_set[:,0:4],training_set[:,-1])

In [82]:
%timeit clf.fit(training_set[:,0:4],training_set[:,-1])

2.5 ms ± 29.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [72]:
f"The test accuracy for sklearn clf is {1-calc_accuracy(test[:,-1],clf.predict(test[:,0:4]))}"


'The test accuracy for sklearn clf is 0.9902912621359223'

In [80]:
# we can see that is slight more accurate that the tree I've trained
# let us see speed differences 
%timeit learn(training_set,'entropy')

3.09 s ± 47.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


243 ns ± 4.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
